In [2]:
import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import pandas as pd  
import numpy as np  
import copy

import tools.Sample_Tools as smpl
from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)


import matplotlib.pyplot as plt
import seaborn as sns

# from base.JuUnits import parallal_task,task_chunk_split
# from base.JuUnits import excute_for_multidates

# from sklearn.decomposition import FactorAnalysis
# from factor_analyzer import FactorAnalyzer
# from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity,calculate_kmo

# import inspect
import warnings
from sklearn.model_selection import train_test_split

from catboost import CatBoost
from catboost import CatBoostRanker, Pool, MetricVisualizer,CatBoostRegressor


In [ ]:
file_names = ['base10','alpha42','alpha54','alpha60','alpha77','191_alpha004', '191_alpha010', '191_alpha021', '191_alpha070', '191_alpha095', '191_alpha115', 
 '191_alpha124', '191_alpha126', '191_alpha129', '191_alpha132', '191_alpha143', '191_alpha144', 
  '191_alpha146', '191_alpha150', '191_alpha153', '191_alpha160', '191_alpha161', '191_alpha173', '191_alpha187']


temp = []
for file in file_names[0:7]:
    print(file)
    temp.append(load_cache(file+'_train',cache_type=CACHE_TYPE.FACTOR)
)

data_all = pd.concat(temp,axis=1).sort_index()#.loc[(slice('2017-01-11','2020-12-24'),codes),:]



base10
alpha42
alpha54
alpha60
alpha77


In [62]:
data_slice = data_all.loc[(slice('2018-01-11','2020-12-24'),slice(None))]


In [63]:
codes = data_slice.index.get_level_values(1).unique()
dates = data_slice.index.get_level_values(0).unique()
data_replenish = load_cache_adv('all_train_qfq',
                                dates.min().strftime('%Y-%m-%d'),
                                dates.max().strftime('%Y-%m-%d'),
                                cache_type=CACHE_TYPE.STOCK)

data_replenish = data_replenish.loc[(slice(None),data_replenish.index.get_level_values(1).unique().intersection(codes)),]

ret = smpl.get_forward_return(data_replenish,'close')
ret.name = 'target'
ret[np.abs(ret)>0.15]=np.nan

industy = data_replenish['industry']
industy.name = 'industy'

cap_log = np.log(data_replenish['market_value'])
cap_log.name = 'cap_log'


data = pd.concat([data_slice,industy,cap_log,ret],axis=1).sort_index().dropna()



In [64]:
data['industy'] = data['industy'].astype('string')
data['191_alpha004'] = data['191_alpha004'].astype('int')
train_data = data[data.columns[:-1]]
train_target = data[[data.columns[-1]]]


X_train,X_test, y_train, y_test =train_test_split(train_data,train_target,test_size=.2, shuffle=False)

X_train_group = y_train.index.get_level_values(0).strftime('%Y-%m-%d').tolist()
y_test_group = y_test.index.get_level_values(0).strftime('%Y-%m-%d').tolist()

In [7]:
# y_train.groupby(level=0).count()
# X_train.nunique()

alpha42     589594
industy        336
cap_log    1053642
dtype: int64

In [46]:
# categorical_features_names = ['191_alpha004', 'industy']
categorical_features_names = ['industy']
train_p = Pool(
    data=X_train,
    label=y_train,
    group_id=X_train_group,
    cat_features=categorical_features_names
)

test_p = Pool(
    data=X_test,
    label=y_test,
    group_id=y_test_group,
    cat_features=categorical_features_names
)

In [66]:
default_parameters = {
    'learning_rate': .05,
    'iterations': 100,
    'custom_metric': None,
    'verbose': False,
    'random_seed': 0,
    'use_best_model':True,
    'early_stopping_rounds': 10,
}



def fit_model(model_class ,loss_function, train_pool, test_pool, additional_params=None, verbose=False):
    parameters = copy.deepcopy(default_parameters)
    parameters['loss_function'] = loss_function
    parameters['train_dir'] = loss_function
    parameters['verbose'] = verbose
    
    if additional_params is not None:
        parameters.update(additional_params)
        
    if model_class == CatBoost:
        mod = model_class(parameters)
    else:
        mod = model_class(**parameters)
        
    mod.fit(train_pool, eval_set=test_pool, plot=True)
    
    return mod



In [70]:
# parameters = {'custom_metric': ['PrecisionAt:top=10', 'RecallAt:top=10', 'MAP:top=10']}
# parameters = {'custom_metric': ['PFound',  'AverageGain:top=10'],}
# parameters = {'custom_metric': ['NDCG',  'AverageGain:top=100'],}
# parameters={'learning_rate': .1,'custom_metric': ['NDCG:type=ranking',  'AverageGain:top=1000'],'eval_metric':'AUC:type=Ranking','one_hot_max_size': X_train['industy'].nunique()}
# parameters={'learning_rate': .1,'custom_metric': ['NDCG', 'AverageGain:top=1000'],'one_hot_max_size': 0}
parameters={'learning_rate': .5,'custom_metric': ['R2', 'AverageGain:top=100'],'one_hot_max_size': 0,'early_stopping_rounds':None,'iterations': 1000,'use_best_model':False}

model = fit_model(CatBoostRegressor, 'RMSE', train_p, test_p, parameters, verbose=False)

# class LoglossObjective(object):
#     # L​(θ;X,y)=1− 1/N​*∑​max(μi​−yi​,0)**2
#     def calc_ders_range(self, approxes, targets, weights):
#         assert len(approxes) == len(targets)
#         if weights is not None:
#             assert len(weights) == len(approxes)

#         result = []
#         for index in range(len(targets)):
#             e = np.exp(approxes[index])
#             p = e / (1 + e)
#             der1 = targets[index] - p
#             der2 = -p * (1 - p)

#             if weights is not None:
#                 der1 *= weights[index]
#                 der2 *= weights[index]

#             result.append((der1, der2))
#         return result

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [71]:
# for i in range(10):
#     print(model.predict(Pool(X_test.iloc[i], cat_features=categorical_features_names)),y_test.iloc[i].values[0])

    
    

y_pred = model.predict(data=X_test,prediction_type='RawFormulaVal')   #RawFormulaVal,Probability, LogProbability
# rank = y_pred.argsort()
# y_pred

(np.sign(y_pred) == np.sign(y_test.values.squeeze())).sum()/len(y_pred)

0.4926030115744528

In [72]:
# y_test.iloc[200:500].values.squeeze()
y_pred = model.predict(data=X_train,prediction_type='RawFormulaVal')   #RawFormulaVal,Probability, LogProbability
(np.sign(y_pred) == np.sign(y_train.values.squeeze())).sum()/len(y_pred)

0.5261678691500984

ValueError: x and y must be the same size

In [19]:
plt.figure(figsize=(1120/72,420/72))
plt.scatter(range(len(y_pred)), np.sign(y_pred),linewidths=4)
plt.scatter(range(len(y_pred)), np.sign(y_test.iloc[700:900].values.squeeze()))


In [333]:
# model.get_feature_importance(train_p)
# feature_importances = model.get_feature_importance(train_p)
# for score, name in sorted(zip(feature_importances, X_train.columns), reverse=True):
#     print('{}: {}'.format(name, score))
######
# widget = MetricVisualizer(['model_depth_5', 'model_depth_8']) #事后对比，model_depth_5是‘train_dir’
# widget.start()
####
# model.save_model('catboost_model.dump')
# model = CatBoostClassifier()
# model.load_model('catboost_model.dump');

########Batch_training
# model1 = CatBoostRegressor(**params)
# batch1 = Pool(train_data1, label=train_labels1)
# model1.fit(X=batch1)
# # continue training with different portion of data
# model2 = CatBoostRegressor(**params)
# batch2 = Pool(train_data2, label=train_labels2)
# batch2.set_baseline(model1.predict(batch2))
# model2.fit(X=batch2)
# # build resulting model
# model = sum_models([model1, model2])



In [396]:
# 'YetiRank' 'RMSE'

parameters={
    'learning_rate': .05,
    'iterations': 50,
    'custom_metric': ['NDCG', 'AverageGain:top=1000'],
    'verbose': False,
    'random_seed': 0,
    'use_best_model':True,
    'early_stopping_rounds': 10,
    'one_hot_max_size': 0
}
# model = fit_model(CatBoostRanker, 'YetiRank', train_p, test_p, parameters, verbose=False)


x = CatBoostRanker(**parameters)
x.fit(train_p, eval_set=test_p, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [399]:
y = x.predict(X_test.iloc[0:10])  
y

array([-0.02283442,  0.04129997,  0.00506462,  0.0366282 , -0.02233261,
        0.07942477,  0.01678374,  0.04442451,  0.00612589, -0.03088151])

In [403]:
y.argsort().argsort()

array([1, 7, 3, 6, 2, 9, 5, 8, 4, 0], dtype=int64)

In [ ]:
array([1, 7, 0, 2, 3, 9, 6, 8, 5, 4], dtype=int64)

In [402]:
y_test.iloc[0:10].values.squeeze().argsort().argsort()

array([0, 2, 3, 4, 6, 1, 9, 5, 7, 8], dtype=int64)